# Restims

In [1]:
import os 
import pandas as pd
import numpy as np
import re

root = "c:\\Users\\Tony\\OneDrive - Australian National University\\Documents - Parkinson's data_\\Harmonised UMAPS"

In [2]:
restim_features = pd.read_csv(
    root + "\\analysis_rawstats\\restim_tcells_rawstats_geomeans.csv"
)[:-2].iloc[:, :-1]
#Flowjo is dumb and can't average 0
restim_features.loc[restim_features["Mean (*Condition :: null)"] == "1.09E?3", "Mean (*Condition :: null)"] = 0
# Need to change the value *again*, for week 30. I have no idea why
restim_features.loc[restim_features["Mean (*Week :: null)"] == 30, "Mean (*Week :: null)"] = 30

restim_metadata = restim_features[["Unnamed: 0", 'Mean (*Concentration(low=0,high=1,DMSO=2) :: null)',
                 'Mean (*Condition :: null)', 'Mean (*MouseID# :: null)', 
                 'Mean (*Strain(CR=0,JAX=1) :: null)', 'Mean (*Week :: null)']].copy()

restim_features = restim_features.drop(["Unnamed: 0", 'Mean (*Concentration(low=0,high=1,DMSO=2) :: null)',
                 'Mean (*Condition :: null)', 'Mean (*MouseID# :: null)', 
                 'Mean (*Strain(CR=0,JAX=1) :: null)', 'Mean (*Week :: null)'], axis = 1)

for column in restim_features.columns:
    restim_features[column] = pd.to_numeric(restim_features[column], errors='coerce').fillna(0)

restim_metadata["Mean (*Condition :: null)"] = pd.to_numeric(restim_metadata["Mean (*Condition :: null)"], errors='coerce').fillna(0)

In [3]:
''' 
Going to split the dataframe by week as a new dataframe. With new dataframe, 
going to change the column label to denote that it is an observation taken at that time. 
Finally, going to reconcile all three time points together where each mouse (identified by mouseid and batch number) 
has a row of events
''' 

def extract_info(s):
    batch_match = re.search(r'_batch (\d+\.\d)', s)
    mouseid_match = re.search(r'_mouseid (\d+\.\d)', s)
    timepoint_match = re.search(r'_week(\d+)', s)
    
    batch = float(batch_match.group(1)) if batch_match else None
    mouseid = float(mouseid_match.group(1)) if mouseid_match else None
    timepoint = f"week{timepoint_match.group(1)}" if timepoint_match else None
    
    # Returning as integers for batch and mouseid for consistency with previous usage
    batch = int(batch) if batch is not None else None
    mouseid = int(mouseid) if mouseid is not None else None
    
    return batch, mouseid, timepoint

# Apply the function and expand the dataframe
restim_metadata[['Batch', 'MouseID', 'Timepoint']] = restim_metadata.apply(
    lambda row: extract_info(row['Unnamed: 0']), axis=1, result_type='expand')

In [4]:
#timepoints = restim_metadata['Timepoint'].unique()
timepoints = ["week0", "week7"]
restim_featuress = {tp: restim_features[restim_metadata['Timepoint'] == tp].copy() for tp in timepoints}
for tp, restim_features_tp in restim_featuress.items():
    new_columns = {col: f"{col}_{tp}" for col in restim_features_tp.columns if col not in ['Batch', 'MouseID', 'Timepoint']}
    restim_features_tp.rename(columns=new_columns, inplace=True)

In [5]:
for tp, df in restim_featuress.items():
    df['MouseID'] = restim_metadata.loc[df.index, 'MouseID']
    df['Batch'] = restim_metadata.loc[df.index, 'Batch']

final_features_df = pd.DataFrame()

# Iterate through the dictionary of enhanced DataFrames to merge them
for tp, features_df in restim_featuress.items():
    # Ensure only feature columns are included in the merge
    feature_columns = [col for col in features_df.columns if col not in ['MouseID', 'Batch', 'Timepoint']]
    if final_features_df.empty:
        final_features_df = features_df
    else:
        # Merge on 'MouseID' and 'Batch', combining features horizontally
        final_features_df = pd.merge(final_features_df, features_df[feature_columns + ['MouseID', 'Batch']], on=['MouseID', 'Batch'], how='outer')

In [6]:
# Prepare a list of all timepoints to calculate deltas
# timepoints = list(restim_featuress.keys())
timepoints = ["week0", "week7"]

# Prepare a dictionary to hold delta DataFrames
delta_dfs = {}

for i, tp1 in enumerate(timepoints):
    for tp2 in timepoints[i+1:]:
        # Merge the DataFrames for the two timepoints on 'MouseID' and 'Batch' to align mice present in both
        df1 = restim_featuress[tp1]
        df2 = restim_featuress[tp2]
        merged_df = pd.merge(df1, df2, on=['MouseID', 'Batch'], suffixes=(f'_{tp1}', f'_{tp2}'), how='inner')

        # Calculate deltas for each feature present in both timepoints
        for col in df1.columns:
            if col not in ['MouseID', 'Batch', 'Timepoint']:  # Exclude non-feature columns
                # Identify corresponding column in df2
                col_in_df2 = col.replace(f'_{tp1}', f'_{tp2}')
                if col_in_df2 in df2.columns:
                    delta_col_name = f"{col}_delta_{tp1}_vs_{tp2}"
                    merged_df[delta_col_name] = -(merged_df[col] - merged_df[col_in_df2])
        
        # Store the delta features in the delta_dfs dictionary
        delta_features = [col for col in merged_df.columns if "delta" in col]
        delta_dfs[f"{tp1}_vs_{tp2}"] = merged_df[['MouseID', 'Batch'] + delta_features]

# Concatenate the delta features onto the final dataframe
for key, delta_df in delta_dfs.items():
    final_features_df = pd.merge(final_features_df, delta_df, on=['MouseID', 'Batch'], how='left')

C:\Users\Tony\AppData\Local\Temp\ipykernel_41132\2490935078.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[delta_col_name] = -(merged_df[col] - merged_df[col_in_df2])
C:\Users\Tony\AppData\Local\Temp\ipykernel_41132\2490935078.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[delta_col_name] = -(merged_df[col] - merged_df[col_in_df2])
C:\Users\Tony\AppData\Local\Temp\ipykernel_41132\2490935078.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

In [7]:
final_features_df = final_features_df.loc[
    final_features_df["CD4s | Freq. of Parent (%)_week7"].notna(), : 
]

# Now just adding back metadata
final_features_df = pd.merge(
    final_features_df, 
    restim_metadata.groupby(["Batch", "MouseID"]).mean()["Mean (*Condition :: null)"], 
    on = ["Batch", "MouseID"], 
    how = "inner")

final_features_df["Condition"] = final_features_df["Mean (*Condition :: null)"]
final_features_df.drop("Mean (*Condition :: null)", axis = 1, inplace = True)

C:\Users\Tony\AppData\Local\Temp\ipykernel_41132\2778698001.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  restim_metadata.groupby(["Batch", "MouseID"]).mean()["Mean (*Condition :: null)"],


In [8]:
final_features_df.to_csv(root + "/peptide_harmonised_features.csv")

In [9]:
'''

Analysis steps: 

- Box plots for query pops
- GLM for sham vs peptide 

Query pops are: 

CD3lo 
Blob2

'''

import plotly.express as px
import statsmodels.api as sm
import statsmodels
from sklearn.decomposition import PCA
import scipy as sp

condition_0 = final_features_df.loc[final_features_df["Condition"] == 0, :].drop(["MouseID","Batch","Condition"], axis = 1)
condition_1 = final_features_df.loc[final_features_df["Condition"] == 1, :].drop(["MouseID","Batch","Condition"], axis = 1)
condition_2 = final_features_df.loc[final_features_df["Condition"] == 2, :].drop(["MouseID","Batch","Condition"], axis = 1)
condition_3 = final_features_df.loc[final_features_df["Condition"] == 3, :].drop(["MouseID","Batch","Condition"], axis = 1)

In [10]:
from sklearn.model_selection import StratifiedKFold
from boruta import BorutaPy as bp
from sklearn.ensemble import RandomForestClassifier as RFC


kf = StratifiedKFold(n_splits=5)
columns_to_keep = [col for col in condition_3.columns if not col.endswith('_week0')]
condition13 = final_features_df.loc[
    (final_features_df["Condition"] == 1) | 
    (final_features_df["Condition"] == 3), :]
condition13 = condition13.dropna()

rank_accumulator = np.zeros(len(columns_to_keep))

for train_dex, test_dex in kf.split(condition13[columns_to_keep], np.array(condition13["Condition"])):
    x_train, x_test = condition13.iloc[train_dex, :][columns_to_keep], condition13.iloc[test_dex, :][columns_to_keep]
    y_train, y_test = condition13.iloc[train_dex, :]["Condition"], condition13.iloc[test_dex, :]["Condition"]

    rf = RFC(n_jobs=-1, class_weight='balanced', max_depth=5) 
    feature_selector = bp(
        verbose = 2, 
        estimator = rf, 
        n_estimators='auto',
        max_iter = 30
        )
    feature_selector.fit(x_train.values, y_train.values)
    rank_accumulator += feature_selector.ranking_


Iteration: 	1 / 30
Confirmed: 	0
Tentative: 	398
Rejected: 	0
Iteration: 	2 / 30
Confirmed: 	0
Tentative: 	398
Rejected: 	0
Iteration: 	3 / 30
Confirmed: 	0
Tentative: 	398
Rejected: 	0
Iteration: 	4 / 30
Confirmed: 	0
Tentative: 	398
Rejected: 	0
Iteration: 	5 / 30
Confirmed: 	0
Tentative: 	398
Rejected: 	0
Iteration: 	6 / 30
Confirmed: 	0
Tentative: 	398
Rejected: 	0
Iteration: 	7 / 30
Confirmed: 	0
Tentative: 	398
Rejected: 	0
Iteration: 	8 / 30
Confirmed: 	0
Tentative: 	0
Rejected: 	398


BorutaPy finished running.

Iteration: 	9 / 30
Confirmed: 	0
Tentative: 	0
Rejected: 	398
Iteration: 	1 / 30
Confirmed: 	0
Tentative: 	398
Rejected: 	0
Iteration: 	2 / 30
Confirmed: 	0
Tentative: 	398
Rejected: 	0
Iteration: 	3 / 30
Confirmed: 	0
Tentative: 	398
Rejected: 	0
Iteration: 	4 / 30
Confirmed: 	0
Tentative: 	398
Rejected: 	0
Iteration: 	5 / 30
Confirmed: 	0
Tentative: 	398
Rejected: 	0
Iteration: 	6 / 30
Confirmed: 	0
Tentative: 	398
Rejected: 	0
Iteration: 	7 / 30
Confirmed: 	0
Tentati

In [16]:
top_10_features_indices = np.argsort(rank_accumulator)[:5]
top_10_features_mask = np.full(rank_accumulator.shape, False, dtype=bool)
top_10_features_mask[top_10_features_indices] = True
top10_feature_names = np.array([col for col in condition13[columns_to_keep].columns[top_10_features_mask]])

In [17]:
final_meta = final_features_df.dropna()[["Batch", "MouseID", "Condition"]]
final_features_df = final_features_df.dropna().drop(["Batch", "MouseID", "Condition"], axis = 1)

KeyError: "None of [Index(['Batch', 'MouseID', 'Condition'], dtype='object')] are in the [columns]"

In [ ]:
selected_pca = PCA(
    whiten = True, 
)

# HAVE TO WHITEN PROPERLY - WHITEN YOU NITWIT
selected_pca_features = selected_pca.fit_transform(final_features_df[top10_feature_names].dropna()/final_features_df[top10_feature_names].abs().max())

# Feature populations
display(px.scatter_3d(
    x = selected_pca_features[:, 0], 
    y = selected_pca_features[:, 1], 
    z = selected_pca_features[:, 2],
    color = final_meta.loc[final_features_df.dropna().index, "Condition"],
    log_x=False,
))

In [14]:
pd.DataFrame(
    selected_pca.components_, 
    columns = top10_feature_names
).T

,0,1,2,3,4,5,6,7,8,9
CD4 IL17hi | Geometric Mean (FSC-A :: null)_week7,-0.156026,-0.046254,-0.184113,-0.100397,0.124792,-0.311847,-0.519319,-0.691272,-0.252615,-0.073175
GMCSFvhi CD4 | Geometric Mean (FOXP3 :: null)_week7,-0.243258,-0.005033,-0.308298,0.076809,0.110295,-0.210867,0.596985,-0.298244,0.082322,0.575418
GMCSFvhi CD4 | Geometric Mean (GMCSF :: null)_week7,-0.213374,0.279276,0.068776,-0.384794,0.776465,0.335486,0.017774,0.064908,-0.060832,-0.001534
GMCSFvhi CD4 | Geometric Mean (IFNy :: null)_week7,-0.312803,0.352675,-0.382690,-0.353201,-0.390595,0.137731,0.206110,-0.122487,0.233237,-0.472391
SSChi | Geometric Mean (GMCSF :: null)_week7,-0.011457,0.098550,-0.087827,0.530603,-0.072646,0.710735,0.047862,-0.317532,-0.294928,-0.019526
SSChi | Geometric Mean (LD :: null)_week7,0.062543,-0.014382,-0.108360,-0.102618,-0.011551,-0.225624,0.357237,0.193086,-0.833837,-0.249769
CD8 IL17hi | Geometric Mean (GMCSF :: null)_week0_delta_week0_vs_week7,0.256348,0.708493,-0.320645,0.391439,0.156950,-0.294474,-0.163879,0.188798,0.047439,0.013616
GMCSFvhi CD4 | Geometric Mean (CD3 :: null)_week0_delta_week0_vs_week7,-0.371378,0.362153,0.736726,0.212271,-0.080352,-0.237280,0.172145,-0.183049,-0.020482,-0.129538
GMCSFvhi CD4 | Geometric Mean (FOXP3 :: null)_week0_delta_week0_vs_week7,-0.472635,-0.376052,-0.218230,0.468331,0.313236,-0.147316,0.012203,0.221893,0.158779,-0.416927
GMCSFvhi CD4 | Geometric Mean (IFNy :: null)_week0_delta_week0_vs_week7,-0.584846,0.104925,-0.084937,-0.038929,-0.286674,0.077439,-0.380225,0.396839,-0.247307,0.432232


In [18]:
# Exploring the GMCSF Population
gmcsf_cols = [col for col in final_features_df.columns if "GMCSFvhi CD4" in col and not col.endswith("_week0")]
for col in gmcsf_cols: 
    display(px.box(
        x = final_meta["Condition"], 
        y = (final_features_df[gmcsf_cols]/final_features_df[gmcsf_cols].max())[col],
        points="all",
        title=col
    ))

In [19]:
# Exploring the SSChi Population
sschi_cols = [col for col in final_features_df.columns if "SSChi" in col and not col.endswith("_week0")]
for col in sschi_cols: 
    display(px.box(
        x = final_meta["Condition"], 
        y = (final_features_df[sschi_cols]/final_features_df[sschi_cols].max())[col],
        points="all",
        title=col
    ))

In [20]:
gmcsf_pca = PCA(
    whiten = True, 
)

gmcsf_pca_features = gmcsf_pca.fit_transform((
    final_features_df[gmcsf_cols + sschi_cols].dropna()/final_features_df[gmcsf_cols + sschi_cols].dropna().abs().max()))

# Feature populations
display(px.scatter_3d(
    x = gmcsf_pca_features[:, 0], 
    y = gmcsf_pca_features[:, 1], 
    z = gmcsf_pca_features[:, 2],
    color = final_meta.loc[final_features_df.dropna().index, "Condition"],
    log_x=False,
))

cd13_gmcsf_pca = PCA(
    whiten = True, 
)

cd13_gmcsf_pca_features = cd13_gmcsf_pca.fit_transform(condition13[gmcsf_cols].dropna())


In [21]:
component = 0
sp.stats.brunnermunzel(
    gmcsf_pca_features[final_meta["Condition"] == 1, component],
    gmcsf_pca_features[final_meta["Condition"] == 3, component],
    alternative = "two-sided")  

BrunnerMunzelResult(statistic=-4.6432639879873365, pvalue=0.00018554055814573215)

# Myeloids

In [11]:
import os 
import pandas as pd
import numpy as np
import re

root = "c:\\Users\\Tony\\OneDrive - Australian National University\\Documents - Parkinson's data_\\Harmonised UMAPS"

In [12]:
myeloid_features = pd.read_csv(
    root + "\\analysis_rawstats\\peptide_myeloids_rawstats_geomeans.csv"
)[:-2].iloc[:, :-1]
#Flowjo is dumb and can't average 0
myeloid_features.loc[myeloid_features["Median (*Condition :: null)"] == "1.09E?3", "Median (*Condition :: null)"] = 0
# Need to change the value *again*, for week 30. I have no idea why
myeloid_features.loc[myeloid_features["Mean (*Week :: null)"] == 30, "Mean (*Week :: null)"] = 30

myeloid_metadata = myeloid_features[["Unnamed: 0", 'Median (*Concentration(low=0,high=1,DMSO=2) :: null)',
                 'Median (*Condition :: null)', 'Mean (*MouseID# :: null)', "Median (*Batch :: null)", 
                 'Median (*Strain(CR=0,JAX=1) :: null)', 'Mean (*Week :: null)']].copy()

myeloid_features = myeloid_features.drop(["Unnamed: 0", 'Median (*Concentration(low=0,high=1,DMSO=2) :: null)',
                 'Median (*Condition :: null)', 'Mean (*MouseID# :: null)', "Median (*Batch :: null)", 
                 'Median (*Strain(CR=0,JAX=1) :: null)', 'Mean (*Week :: null)'], axis = 1)

for column in myeloid_features.columns:
    myeloid_features[column] = pd.to_numeric(myeloid_features[column], errors='coerce').fillna(0)

myeloid_metadata["Median (*Condition :: null)"] = pd.to_numeric(myeloid_metadata["Median (*Condition :: null)"], errors='coerce').fillna(0)

In [13]:
''' 
Going to split the dataframe by week as a new dataframe. With new dataframe, 
going to change the column label to denote that it is an observation taken at that time. 
Finally, going to reconcile all three time points together where each mouse (identified by mouseid and batch number) 
has a row of events
''' 

def extract_info(s):
    batch_match = re.search(r'_batch (\d+\.\d)', s)
    mouseid_match = re.search(r'_mouseid (\d+\.\d)', s)
    timepoint_match = re.search(r'_week(\d+)', s)
    
    batch = float(batch_match.group(1)) if batch_match else None
    mouseid = float(mouseid_match.group(1)) if mouseid_match else None
    timepoint = f"week{timepoint_match.group(1)}" if timepoint_match else None
    
    # Returning as integers for batch and mouseid for consistency with previous usage
    batch = int(batch) if batch is not None else None
    mouseid = int(mouseid) if mouseid is not None else None
    
    return batch, mouseid, timepoint

# Apply the function and expand the dataframe
myeloid_metadata[['Batch', 'MouseID', 'Timepoint']] = myeloid_metadata.apply(
    lambda row: extract_info(row['Unnamed: 0']), axis=1, result_type='expand')

In [14]:
#timepoints = myeloid_metadata['Timepoint'].unique()
timepoints = ["week0", "week1", "week7"]
myeloid_featuress = {tp: myeloid_features[myeloid_metadata['Timepoint'] == tp].copy() for tp in timepoints}
for tp, myeloid_features_tp in myeloid_featuress.items():
    new_columns = {col: f"{col}_{tp}" for col in myeloid_features_tp.columns if col not in ['Batch', 'MouseID', 'Timepoint']}
    myeloid_features_tp.rename(columns=new_columns, inplace=True)

In [15]:
for tp, df in myeloid_featuress.items():
    df['MouseID'] = myeloid_metadata.loc[df.index, 'MouseID']
    df['Batch'] = myeloid_metadata.loc[df.index, 'Batch']

myeloid_final_features_df = pd.DataFrame()

# Iterate through the dictionary of enhanced DataFrames to merge them
for tp, features_df in myeloid_featuress.items():
    # Ensure only feature columns are included in the merge
    feature_columns = [col for col in features_df.columns if col not in ['MouseID', 'Batch', 'Timepoint']]
    if myeloid_final_features_df.empty:
        myeloid_final_features_df = features_df
    else:
        # Merge on 'MouseID' and 'Batch', combining features horizontally
        myeloid_final_features_df = pd.merge(myeloid_final_features_df, features_df[feature_columns + ['MouseID', 'Batch']], on=['MouseID', 'Batch'], how='outer')


In [16]:
# Prepare a list of all timepoints to calculate deltas
# timepoints = list(myeloid_featuress.keys())
timepoints = ["week0", "week1", "week7"]

# Prepare a dictionary to hold delta DataFrames
delta_dfs = {}

for i, tp1 in enumerate(timepoints):
    for tp2 in timepoints[i+1:]:
        # Merge the DataFrames for the two timepoints on 'MouseID' and 'Batch' to align mice present in both
        df1 = myeloid_featuress[tp1]
        df2 = myeloid_featuress[tp2]
        merged_df = pd.merge(df1, df2, on=['MouseID', 'Batch'], suffixes=(f'_{tp1}', f'_{tp2}'), how='inner')

        # Calculate deltas for each feature present in both timepoints
        for col in df1.columns:
            if col not in ['MouseID', 'Batch', 'Timepoint']:  # Exclude non-feature columns
                # Identify corresponding column in df2
                col_in_df2 = col.replace(f'_{tp1}', f'_{tp2}')
                if col_in_df2 in df2.columns:
                    delta_col_name = f"{col}_delta_{tp1}_vs_{tp2}"
                    merged_df[delta_col_name] = -(merged_df[col] - merged_df[col_in_df2])
        
        # Store the delta features in the delta_dfs dictionary
        delta_features = [col for col in merged_df.columns if "delta" in col]
        delta_dfs[f"{tp1}_vs_{tp2}"] = merged_df[['MouseID', 'Batch'] + delta_features]

# Concatenate the delta features onto the final dataframe
for key, delta_df in delta_dfs.items():
    myeloid_final_features_df = pd.merge(myeloid_final_features_df, delta_df, on=['MouseID', 'Batch'], how='left')

C:\Users\Tony\AppData\Local\Temp\ipykernel_41132\2085180950.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[delta_col_name] = -(merged_df[col] - merged_df[col_in_df2])
C:\Users\Tony\AppData\Local\Temp\ipykernel_41132\2085180950.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[delta_col_name] = -(merged_df[col] - merged_df[col_in_df2])
C:\Users\Tony\AppData\Local\Temp\ipykernel_41132\2085180950.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

In [17]:
# Now just adding back metadata
myeloid_final_features_df = pd.merge(
    myeloid_final_features_df, 
    myeloid_metadata.groupby(["Batch", "MouseID"]).mean()["Median (*Condition :: null)"], 
    on = ["Batch", "MouseID"], 
    how = "inner")

myeloid_final_features_df["Condition"] = myeloid_final_features_df["Median (*Condition :: null)"]

# Aggregating Conditions

myeloid_final_features_df.drop("Median (*Condition :: null)", axis = 1, inplace = True)

C:\Users\Tony\AppData\Local\Temp\ipykernel_41132\1011818548.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  myeloid_metadata.groupby(["Batch", "MouseID"]).mean()["Median (*Condition :: null)"],


In [19]:
'''

Analysis steps: 

- Box plots for query pops
- GLM for sham vs peptide 

Query pops are: 

CD3lo 
Blob2

'''

import plotly.express as px
import statsmodels.api as sm
import statsmodels
import scipy as sp

# Dropping all the bad batches
myeloid_final_features_df.to_csv(root + "/myeloid_harmonised_features.csv")
myeloid_final_features_df_full = myeloid_final_features_df.copy()
myeloid_final_features_df = myeloid_final_features_df.loc[myeloid_final_features_df["Batch"] == 10]
# myeloid_final_features_df = myeloid_final_features_df.loc[
#     (myeloid_final_features_df["Batch"] == 8) | (myeloid_final_features_df["Batch"] == 10), :]

myeloid_condition_0 = myeloid_final_features_df.loc[myeloid_final_features_df["Condition"] == 0, :].drop(["Condition", "Batch", "MouseID"], axis = 1)
myeloid_condition_1 = myeloid_final_features_df.loc[myeloid_final_features_df["Condition"] == 1, :].drop(["Condition", "Batch", "MouseID"], axis = 1)
myeloid_condition_2 = myeloid_final_features_df.loc[myeloid_final_features_df["Condition"] == 2, :].drop(["Condition", "Batch", "MouseID"], axis = 1)
myeloid_condition_3 = myeloid_final_features_df.loc[myeloid_final_features_df["Condition"] == 3, :].drop(["Condition", "Batch", "MouseID"], axis = 1)

In [65]:
def modify_conditions(condition): 
    if condition in (3,5): 
        return True
    else:
        return False
    
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5)
myeloid_columns_to_keep = [
    col for col in myeloid_condition_3.columns if
    #   not col.endswith('_week0') and "Condition" not in col and "umapneg_b220hi" not in col]
      not col.endswith('_week0') and "Condition" not in col]
myeloid_final_features_df.dropna(inplace = True)
modified_condition = myeloid_final_features_df["Condition"].apply(modify_conditions)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_myeloid_final_features_df = scaler.fit_transform(myeloid_final_features_df[myeloid_columns_to_keep])

# Batch correcting with GLM
# batch_adjusted_myeloid = np.zeros_like(myeloid_final_features_df)

# for i, column in enumerate(myeloid_final_features_df[myeloid_columns_to_keep].columns):
#     y = scaled_myeloid_final_features_df[:, i]
#     # Get dummy variables for batch. Avoiding the dummy variable trap by dropping first
#     X = pd.get_dummies(myeloid_final_features_df["Batch"], drop_first=True)
#     X = sm.add_constant(X)  # Adds a constant term to the predictor
    
#     # Fit the GLM
#     # 'family' is chosen based on the nature of the dependent variable 'y'
#     # E.g., if 'y' is count data, you might choose family=sm.families.Poisson()
#     # In this case, we'll use the normal distribution as an example
#     ols = sm.GLM(y, X, family=sm.families.Binomial()).fit()
    
#     # Subtract the effect of the batch
#     batch_adjusted_myeloid[:, i] = y - glm.mu

# # Batch Correcting further
# myeloid_norm_features_batch_wise = pd.DataFrame()

# for batch in myeloid_final_features_df['Batch'].unique():
#     batch_df = myeloid_final_features_df[myeloid_final_features_df['Batch'] == batch]
#     batch_norm = (batch_df[myeloid_columns_to_keep].dropna() + 0.0001) / (batch_df[myeloid_columns_to_keep].dropna().abs().max() + 0.0001)
#     myeloid_norm_features_batch_wise = pd.concat([myeloid_norm_features_batch_wise, batch_norm])

# myeloid_final_features_df[myeloid_columns_to_keep] = myeloid_norm_features_batch_wise.copy(deep = True).dropna()

In [68]:
# Assuming myeloid_columns_to_keep and other necessary variables are already defined
myeloid_confirmed_importance_accumulator = np.zeros(len(myeloid_columns_to_keep))
myeloid_tentative_importance_accumulator = np.zeros(len(myeloid_columns_to_keep))
myeloid_rank_accumulator = np.zeros(len(myeloid_columns_to_keep))

for train_dex, test_dex in kf.split(myeloid_final_features_df[myeloid_columns_to_keep], np.array(modified_condition)):
    x_train, x_test = myeloid_final_features_df.iloc[train_dex, :][myeloid_columns_to_keep], myeloid_final_features_df.iloc[test_dex, :][myeloid_columns_to_keep]
    y_train, y_test = myeloid_final_features_df.iloc[train_dex, :]["Condition"], myeloid_final_features_df.iloc[test_dex, :]["Condition"]

    rf = RFC(n_jobs=-1, class_weight='balanced', max_depth=5)
    feature_selector = bp(
        verbose=2,
        estimator=rf,
        n_estimators='auto',
        max_iter=30
    )
    feature_selector.fit(x_train.values, y_train.values)

    # Update the accumulators based on the support arrays
    myeloid_confirmed_importance_accumulator += feature_selector.support_.astype(int)
    myeloid_tentative_importance_accumulator += feature_selector.support_weak_.astype(int)
    myeloid_rank_accumulator += feature_selector.ranking_

# Now, myeloid_confirmed_importance_accumulator and myeloid_tentative_importance_accumulator contain counts of how often features are confirmed or tentatively important.

Iteration: 	1 / 30
Confirmed: 	0
Tentative: 	1070
Rejected: 	0
Iteration: 	2 / 30
Confirmed: 	0
Tentative: 	1070
Rejected: 	0
Iteration: 	3 / 30
Confirmed: 	0
Tentative: 	1070
Rejected: 	0
Iteration: 	4 / 30
Confirmed: 	0
Tentative: 	1070
Rejected: 	0
Iteration: 	5 / 30
Confirmed: 	0
Tentative: 	1070
Rejected: 	0
Iteration: 	6 / 30
Confirmed: 	0
Tentative: 	1070
Rejected: 	0
Iteration: 	7 / 30
Confirmed: 	0
Tentative: 	1070
Rejected: 	0
Iteration: 	8 / 30
Confirmed: 	0
Tentative: 	0
Rejected: 	1070


BorutaPy finished running.

Iteration: 	9 / 30
Confirmed: 	0
Tentative: 	0
Rejected: 	1070
Iteration: 	1 / 30
Confirmed: 	0
Tentative: 	1070
Rejected: 	0
Iteration: 	2 / 30
Confirmed: 	0
Tentative: 	1070
Rejected: 	0
Iteration: 	3 / 30
Confirmed: 	0
Tentative: 	1070
Rejected: 	0
Iteration: 	4 / 30
Confirmed: 	0
Tentative: 	1070
Rejected: 	0
Iteration: 	5 / 30
Confirmed: 	0
Tentative: 	1070
Rejected: 	0
Iteration: 	6 / 30
Confirmed: 	0
Tentative: 	1070
Rejected: 	0
Iteration: 	7 / 30
Confir

In [75]:
# myeloid_mask = myeloid_confirmed_importance_accumulator != 0
myeloid_mask = (myeloid_tentative_importance_accumulator != 0) | (myeloid_confirmed_importance_accumulator != 0)
myeloid_selected_feature_names = np.array([col for col in myeloid_final_features_df[myeloid_columns_to_keep].columns[myeloid_mask]])
myeloid_selected_feature_names

array(['CD11chi | Geometric Mean (SSC-A :: null)_week1',
       'blob3 | Geometric Mean (FSC-A :: null)_week0_delta_week0_vs_week7',
       'Ly6Gvhi | Geometric Mean (CD11C :: null)_week0_delta_week0_vs_week7',
       'Ly6Gvhi | Geometric Mean (SSC-A :: null)_week0_delta_week0_vs_week7',
       'CD11bhi | Geometric Mean (CD11B :: null)_week1_delta_week1_vs_week7',
       'LDlo | Geometric Mean (CD11B :: null)_week1_delta_week1_vs_week7',
       'Ly6Gvhi | Geometric Mean (CD11C :: null)_week1_delta_week1_vs_week7'],
      dtype='<U67')

In [70]:
myeloid_top_10_features_indices = np.argsort(myeloid_mask)
myeloid_top_10_features_mask = np.full(myeloid_rank_accumulator.shape, False, dtype=bool)
myeloid_top_10_features_mask[myeloid_top_10_features_indices] = True
myeloid_top10_feature_names = np.array([col for col in myeloid_final_features_df[myeloid_columns_to_keep].columns[myeloid_top_10_features_mask]])

In [83]:
p_vals = [
    sp.stats.brunnermunzel(
        myeloid_final_features_df.loc[modified_condition, col],
        myeloid_final_features_df.loc[~modified_condition, col],
        alternative = "two-sided",
        )[1] for col in myeloid_selected_feature_names]
adj_pvals = statsmodels.stats.multitest.multipletests(
    p_vals, 
    method = "fdr_tsbky"
)
adj_pvals

(array([False, False, False, False,  True,  True, False]),
 array([9.78422600e-02, 1.25650756e-01, 1.28148432e-01, 6.01244463e-01,
        2.14563580e-05, 2.14563580e-05, 2.39386634e-01]),
 0.007300831979014655,
 0.0071428571428571435)

In [84]:
from sklearn.decomposition import PCA
pca = PCA(
    whiten = True, 
)
myeloid_final_features_df_batch = myeloid_final_features_df_full.loc[myeloid_final_features_df_full["Batch"] == 10, :]

myeloid_norm_features = (
    myeloid_final_features_df_batch[myeloid_selected_feature_names].dropna() + 0.0001) / (
        myeloid_final_features_df_batch[myeloid_selected_feature_names].dropna().abs().max() + 0.0001)
pca_features = pca.fit_transform(myeloid_final_features_df_batch[myeloid_selected_feature_names].dropna())

# Feature populations
display(px.scatter_3d(
    x = pca_features[:, 0], 
    y = pca_features[:, 1], 
    z = pca_features[:, 2],
    # color = myeloid_final_features_df_batch["Batch"].dropna(),
    color = myeloid_final_features_df_batch.dropna()["Condition"].apply(modify_conditions),
))